In [51]:
from bs4 import BeautifulSoup as bs
import urllib2
import Queue
import sqlite3

In [20]:
doc_name = 'sites.html'

with open(doc_name) as fp:
    soup = bs(fp, 'html.parser')
# print(soup.prettify())

In [23]:
valid_links = [link.get('href') for link in soup.find_all('a') if not 'bible-quiz-index' in str(link)]
for i in range(len(valid_links)):
    print(i, valid_links[i])

(0, u'http://www.religionresourcesonline.org/bible-quiz/print-52-women-in-the-old-testament-bible-quiz')
(1, u'http://www.religionresourcesonline.org/bible-quiz/answers-52-women-in-the-old-testament-bible-quiz')
(2, u'http://www.religionresourcesonline.org/bible-quiz/print-33-bible-quiz-who-said-that-new-testament-2')
(3, u'http://www.religionresourcesonline.org/bible-quiz/answers-33-bible-quiz-who-said-that-new-testament-2')
(4, u'http://www.religionresourcesonline.org/bible-quiz/print-3-bible-quiz-the-creation')
(5, u'http://www.religionresourcesonline.org/bible-quiz/answers-3-bible-quiz-the-creation')
(6, u'http://www.religionresourcesonline.org/bible-quiz/print-9-bible-quiz-noah-and-the-ark')
(7, u'http://www.religionresourcesonline.org/bible-quiz/answers-9-bible-quiz-noah-and-the-ark')
(8, u'http://www.religionresourcesonline.org/bible-quiz/print-302-bible-quiz-on-luke')
(9, u'http://www.religionresourcesonline.org/bible-quiz/answers-302-bible-quiz-on-luke')
(10, u'http://www.reli

In [52]:
class Question:
    def __init__(self, question, ref, num, parts):
        self.question = question
        self.ref = ref
        self.num = num
        self.parts = parts
        self.ans = 'None'
        
    def add_ans(self, ans):
        self.ans = ans
        
    def __str__(self):
        q = ""
        q += "question: " + self.question +"\n"
        q += "ref: "+self.ref + "\n"
        q += "parts: "+ str(self.parts)+"\n"
        q += "ans: " + str(self.ans) + "\n"
        return q

In [6]:
for link in valid_links:
    page = urllib2.urlopen(link).read()
    soup = bs(page, 'html.parser')

In [53]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
 
    return None

In [68]:
def create_rec(conn, q):
    sql = ''' INSERT INTO questions(q_text, ref, num, ops, ans) VALUES(?,?,?,?,?) '''
    cur = conn.cursor()
    question_data = (q.question, q.ref, str(q.num), str(q.parts), q.ans)
    cur.execute(sql, question_data)
    return cur.lastrowid
    

In [55]:
question_link = urllib2.urlopen(valid_links[36]).read()
question_html = bs(question_link, 'html.parser')

# question_parts = []
# for li in question_html.find_all('li', 'question'):
#     for ul in li.find_all('ul', 'question-text'):
#         question_parts.append([text.encode('ascii', 'ignore') for text in ul.stripped_strings])
#         print(ul.text)    
#         print(ul.find_all('span'))

In [66]:
db_file = '../trivia.db'
conn = create_connection(db_file)

In [71]:
queue = Queue.Queue()
for i in range(36, 38, 2):
    question_link = urllib2.urlopen(valid_links[i]).read()
    question_html = bs(question_link, 'html.parser')
    answer_link = urllib2.urlopen(valid_links[i+1]).read()
    answer_html = bs(answer_link, 'html.parser')
    
    for li in question_html.find_all('li', 'question'):
        for ul in li.find_all('ul', 'question-text'):
            parts = [text.encode('ascii', 'ignore') for text in ul.stripped_strings]
            ques = parts.pop(0)
            ref = 'None'
            if ul.find_all('span') != []:
                ref = parts.pop(0)
            queue.put(Question(ques, ref, len(parts), parts))
    
    answer_parts = []
    for li in answer_html.find_all('li', 'question'):
        for ul in li.find_all('ul', 'question-text'):
            answer_parts.append([text.encode('ascii', 'ignore') for text in ul.stripped_strings])

    
    for j in range(len(answer_parts)):
        q = queue.get()
        if q.ref == 'None':
            q.add_ans(answer_parts[j][1])
        else:
            q.add_ans(answer_parts[j][2])
#         print(q)
        print(create_rec(conn, q))
#     for question in question_parts:
#         print(question)
    conn.commit()

61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80


In [72]:
conn.close()